# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
#### >>> Helper Section <<< ####

# help(os)

In [2]:
## Write your code below.

# load dotenv library with magic command "%"
%load_ext dotenv
# load the environment variables
%dotenv 

In [3]:
import os, sys
sys.path.append(os.getenv('SRC_DIR'))


In [4]:
## start logger 
from utils.logger import get_logger
_logs = get_logger(__name__)

from datetime import datetime
now = datetime.now()
formatted_date_weekday = now.strftime("%A, %B %d, %Y")
_logs.info(f"Hello! Welcome to Python's World 🐍🐍~! Today is {datetime.now().strftime('%A, %B %d, %Y')}")

2025-10-02 14:46:41,109, 1150614155.py, 8, INFO, Hello! Welcome to Python's World 🐍🐍~! Today is Thursday, October 02, 2025


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [5]:
## Write your code below.

Path_Price_Data = os.getenv('PRICE_DATA')

from glob import glob

parquet_files = glob(os.path.join(Path_Price_Data, "**/*.parquet"), recursive = True)
parquet_files

['../../05_src/data/prices/PLG/PLG_2012/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2012/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2015/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2015/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2017/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2017/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2019/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2019/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2007/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2007/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2009/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2009/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2006/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2006/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2018/part.1.parquet',
 '../../05_src/data/prices/PLG/PLG_2018/part.0.parquet',
 '../../05_src/data/prices/PLG/PLG_2011/part.1.parquet',
 '../../05_src/data/prices/PLG/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
## Write your code below.
import dask.dataframe as dd
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

In [7]:
dd_px


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=60,,,,,,,,,
ACBI,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ACM,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZN,...,...,...,...,...,...,...,...,...
ZN,...,...,...,...,...,...,...,...,...


In [8]:
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')

In [9]:
# add new column Close_lag_1
dd_shift = dd_px.groupby('ticker',group_keys=False).apply(
    lambda x:x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1))
)

/tmp/ipykernel_24465/1401413171.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker',group_keys=False).apply(


In [10]:
# calculate Return as ROI_pct 
dd_roi = dd_shift.assign(
    ROI_pct = lambda x: (x['Close']/x['Close_lag_1'] - 1)*100
)

# calcualte hi_low-range
dd_roi_HL = dd_roi.assign(
    hi_lo_range = lambda x: x['High']-x['Low']
)

In [11]:
# Check the output by looking at the first 10 row from MET 
met = dd_roi_HL.loc['MET']
met.compute().head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,ROI_pct,hi_lo_range
ticker,,,,,,,,,,,,,
MET,2000-04-05,12.923351,14.037434,12.756239,13.814617,8.877910,77084700.0,MET.csv,2000,NaN,NaN,NaN,1.281195
MET,2000-04-06,13.703209,14.148841,13.591801,13.870321,8.913709,17167700.0,MET.csv,2000,13.814617,8.877910,0.403226,0.557040
MET,2000-04-07,13.703209,13.870321,13.480392,13.647505,8.770515,15131500.0,MET.csv,2000,13.870321,8.913709,-1.606426,0.389929
MET,2000-04-10,13.758913,13.814617,13.591801,13.758913,8.842111,6324900.0,MET.csv,2000,13.647505,8.770515,0.816327,0.222816
MET,2000-04-11,13.647505,13.703209,13.424688,13.480392,8.663124,6087800.0,MET.csv,2000,13.758913,8.842111,-2.024292,0.278521
MET,2000-04-12,13.591801,14.928699,13.480392,14.427361,9.271688,12169800.0,MET.csv,2000,13.480392,8.663124,7.024788,1.448306
MET,2000-04-13,14.483066,15.374331,14.204545,15.151515,9.737061,11922400.0,MET.csv,2000,14.427361,9.271688,5.019307,1.169786
MET,2000-04-14,15.095811,15.151515,14.093137,14.148841,9.092695,10659300.0,MET.csv,2000,15.151515,9.737061,-6.617649,1.058378
MET,2000-04-17,14.260249,14.650178,14.260249,14.371657,9.235890,3951900.0,MET.csv,2000,14.148841,9.092695,1.574804,0.389929


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [12]:
## Write your code below.
## Convert the Dask data frame to a pandas data frame. 

import time
start = time.time() # start time

import tracemalloc 
tracemalloc.start() # start runtime ram tracking

df = dd_roi_HL.compute() 
display(df.head(20))

end = time.time()
_logs.info(f'Performing operation and reading the first 20 rows of df takes {end - start} seconds.')

# object ram 
_logs.info(f"The object 'dd_ROI_HL' when in Dask data frame, occupies {sys.getsizeof(dd_roi_HL)} byte of memory")

# runtime memory checked
current, peak = tracemalloc.get_traced_memory()
_logs.info(f"Runtime Memory Profile - Current: {current / 1024**2:.2f} MB; Peak: {peak / 1024**2:.2f} MB")

tracemalloc.stop()


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,ROI_pct,hi_lo_range
ticker,,,,,,,,,,,,,
ACBI,2015-11-02,13.800,15.000,13.800,15.000,15.000,1700.0,ACBI.csv,2015,NaN,NaN,NaN,1.200000
ACBI,2015-11-03,17.500,17.500,15.660,15.660,15.660,3800.0,ACBI.csv,2015,15.000,15.000,4.399999,1.840000
ACBI,2015-11-04,15.001,15.050,14.300,14.305,14.305,76400.0,ACBI.csv,2015,15.660,15.660,-8.652615,0.750000
ACBI,2015-11-05,14.300,14.500,13.990,14.050,14.050,147000.0,ACBI.csv,2015,14.305,14.305,-1.782594,0.510000
ACBI,2015-11-06,14.160,14.205,14.070,14.150,14.150,51200.0,ACBI.csv,2015,14.050,14.050,0.711740,0.135000
ACBI,2015-11-09,14.190,14.350,14.150,14.250,14.250,25600.0,ACBI.csv,2015,14.150,14.150,0.706716,0.200001
ACBI,2015-11-10,14.600,14.600,14.215,14.250,14.250,16400.0,ACBI.csv,2015,14.250,14.250,0.000000,0.385000
ACBI,2015-11-11,14.250,14.375,14.250,14.300,14.300,186400.0,ACBI.csv,2015,14.250,14.250,0.350879,0.125000
ACBI,2015-11-12,14.300,14.300,14.200,14.260,14.260,33800.0,ACBI.csv,2015,14.300,14.300,-0.279720,0.100000


2025-10-02 14:47:20,929, 2289945448.py, 14, INFO, Performing operation and reading the first 20 rows of df takes 17.600475788116455 seconds.
2025-10-02 14:47:20,930, 2289945448.py, 17, INFO, The object 'dd_ROI_HL' when in Dask data frame, occupies 48 byte of memory
2025-10-02 14:47:20,931, 2289945448.py, 21, INFO, Runtime Memory Profile - Current: 23.15 MB; Peak: 96.97 MB


In [13]:
## add new feature column 10day_SMA for the Return

# start time and runtime ram tracking
start = time.time()
tracemalloc.start()  

dd_roi_HL_rol10 = dd_roi_HL.assign(
    ROI_pct_SMA_10 = lambda x: x['ROI_pct'].rolling(10).mean()
)
# time 
dd_roi_HL_rol10
end = time.time()
_logs.info(f'Performing SMA calculation takes {end - start} seconds.')

# object ram 
_logs.info(f"The object 'dd_roi_HL_rol10' when in Dask data frame, occupies {sys.getsizeof(dd_roi_HL_rol10)} byte of memory")

# runtime memory checked
current, peak = tracemalloc.get_traced_memory()
_logs.info(f"Runtime Memory Profile - Current: {current / 1024**2:.2f} MB; Peak: {peak / 1024**2:.2f} MB")

tracemalloc.stop()

2025-10-02 14:47:20,982, 427478100.py, 13, INFO, Performing SMA calculation takes 0.03949713706970215 seconds.
2025-10-02 14:47:20,984, 427478100.py, 16, INFO, The object 'dd_roi_HL_rol10' when in Dask data frame, occupies 48 byte of memory
2025-10-02 14:47:20,985, 427478100.py, 20, INFO, Runtime Memory Profile - Current: 0.10 MB; Peak: 0.11 MB


In [14]:
# Check the output by looking at the first 10 row from MET 
start = time.time()
met = dd_roi_HL_rol10.loc['MET']
display(met.compute().head(20))

end = time.time()
_logs.info(f'Performing operation and reading the first 20 rows for ticker "MET" takes {end - start} seconds.')

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,ROI_pct,hi_lo_range,ROI_pct_SMA_10
ticker,,,,,,,,,,,,,,
MET,2002-07-03,26.158646,26.158646,25.499109,25.757576,16.826885,3516600.0,MET.csv,2002,NaN,NaN,NaN,0.659536,NaN
MET,2002-07-05,26.069519,26.327986,25.935829,26.203209,17.118010,1525100.0,MET.csv,2002,25.757576,16.826885,1.730104,0.392157,NaN
MET,2002-07-08,26.292336,26.488413,25.935829,26.363636,17.222818,1270300.0,MET.csv,2002,26.203209,17.118010,0.612242,0.552584,NaN
MET,2002-07-09,26.559715,26.648842,25.623886,25.641710,16.751184,1526700.0,MET.csv,2002,26.363636,17.222818,-2.738339,1.024956,NaN
MET,2002-07-10,25.891266,26.024956,24.937611,24.937611,16.291216,2284200.0,MET.csv,2002,25.641710,16.751184,-2.745915,1.087345,NaN
MET,2002-07-11,24.955437,25.525846,24.598930,24.910873,16.273756,1920900.0,MET.csv,2002,24.937611,16.291216,-0.107216,0.926916,NaN
MET,2002-07-12,25.222816,25.222816,24.242424,24.705883,16.139833,1266600.0,MET.csv,2002,24.910873,16.273756,-0.822895,0.980392,NaN
MET,2002-07-15,24.643494,24.643494,23.556150,24.598930,16.069962,2030200.0,MET.csv,2002,24.705883,16.139833,-0.432904,1.087343,NaN
MET,2002-07-16,24.197861,24.866310,23.110518,23.333334,15.243179,2811200.0,MET.csv,2002,24.598930,16.069962,-5.144924,1.755793,NaN


2025-10-02 14:47:30,728, 4115797574.py, 7, INFO, Performing operation and reading the first 20 rows for ticker "MET" takes 9.68906021118164 seconds.


### Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? 

    **Ans**: It depends on the ultimate goal, and the downstream application. If just considering the memory allocated to the object itself and the speed of doing calculation, doing in Dask only take less than 0.01 seconds and the object itself is only occupied 48 byte of memory (as it is just the container).  
      
    In Pandas, just **the object itself will take about 30 MB of memory** and the calculation takes about 0.5 - 0.8 sec. If the memory size is a concern, run in the Dask. It is worth to mention than it will take about than 10 seconds for Dask to the calculation one ticker (in our case above, 'MET') and convert it to pandas data frame. So it would be better to perform all the actual calculation in the end - the concept of **lazy execution**   
    

+ Would it have been better to do it in Dask? Why? 

    **Ans**: As it mentioned previously, If the memory size is a concern, run in the Dask. The lazy execution defers the actual operation in the end. This is commonly used to defer heavy computation particularity in training larger models in ML. 

(1 pt)

In [15]:
##  BONUS: Time when calculate in pandas 
start = time.time()  
tracemalloc.start()

# calculatio in pandas
df_new = df.groupby('ticker', group_keys=False).apply( 
    lambda x: x.assign(ROI_pct_SMA10 = x['ROI_pct'].rolling(10).mean())
)
end = time.time() # end time
_logs.info(f'Performing SMA calculation takes {end - start} seconds.')

# object occupied ram 
_logs.info(f"The object 'df_new' converted to pandas data frame, occupies {sys.getsizeof(df_new)/1024**2:.2f} MB of memory")

# run time ram occupied
current, peak = tracemalloc.get_traced_memory()
_logs.info(f"Runtime Memory Profile - Current: {current / 1024**2:.2f} MB; Peak: {peak / 1024**2:.2f} MB")

display(df_new.loc['MET'].head(20))


2025-10-02 14:47:31,067, 2641134580.py, 10, INFO, Performing SMA calculation takes 0.33185768127441406 seconds.
2025-10-02 14:47:31,224, 2641134580.py, 13, INFO, The object 'df_new' converted to pandas data frame, occupies 31.16 MB of memory
2025-10-02 14:47:31,225, 2641134580.py, 17, INFO, Runtime Memory Profile - Current: 39.07 MB; Peak: 78.23 MB


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,ROI_pct,hi_lo_range,ROI_pct_SMA10
ticker,,,,,,,,,,,,,,
MET,2020-02-19,52.009998,52.160000,51.709999,51.790001,51.790001,5528700.0,MET.csv,2020,NaN,NaN,NaN,0.450001,NaN
MET,2020-02-20,51.840000,52.349998,51.669998,52.279999,52.279999,3686900.0,MET.csv,2020,51.790001,51.790001,0.946124,0.680000,NaN
MET,2020-02-21,51.980000,52.060001,50.959999,51.230000,51.230000,5308000.0,MET.csv,2020,52.279999,52.279999,-2.008415,1.100002,NaN
MET,2020-02-24,49.290001,49.660000,48.799999,49.380001,49.380001,8697000.0,MET.csv,2020,51.230000,51.230000,-3.611162,0.860001,NaN
MET,2020-02-25,49.450001,49.509998,47.279999,47.580002,47.580002,10896800.0,MET.csv,2020,49.380001,49.380001,-3.645199,2.230000,NaN
MET,2020-02-26,47.959999,48.299999,46.730000,46.919998,46.919998,9898400.0,MET.csv,2020,47.580002,47.580002,-1.387145,1.570000,NaN
MET,2020-02-27,45.779999,45.950001,43.799999,43.889999,43.889999,13487100.0,MET.csv,2020,46.919998,46.919998,-6.457798,2.150002,NaN
MET,2020-02-28,42.110001,42.970001,41.470001,42.720001,42.720001,13640200.0,MET.csv,2020,43.889999,43.889999,-2.665751,1.500000,NaN
MET,2020-03-02,42.930000,44.580002,42.709999,44.560001,44.560001,13068400.0,MET.csv,2020,42.720001,42.720001,4.307116,1.870003,NaN


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.